In [ ]:
import boto3

elb = boto3.client('elb')
ec2 = boto3.client('ec2')

In [ ]:
# What are the availability zones in the region?
response = ec2.describe_availability_zones()

zones = []
for z in response['AvailabilityZones']:
    zones.append(z['ZoneName'])
    
print zones

In [ ]:
# Make a web security group for the load balancer
response = ec2.create_security_group(
    GroupName='web_sg',
    Description='SG for web ingres'
)

webSGID = response['GroupId']
print 'created security group ', webSGID

# Allow ingress for ssh
response = ec2.authorize_security_group_ingress(
    GroupId=webSGID,
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
# Create the load balancer
create_elb_response = elb.create_load_balancer(
    LoadBalancerName='sample-lb',
    Listeners=[
        {
            'Protocol': 'HTTP',
            'LoadBalancerPort': 80,
            'InstanceProtocol': 'HTTP',
            'InstancePort': 80
        },
    ],
    AvailabilityZones=[
        zones[0], zones[1]
    ],
    SecurityGroups=[
        webSGID,
    ]
)

print create_elb_response

In [ ]:
response = elb.describe_load_balancers(
    LoadBalancerNames=[
        'sample-lb',
    ]
)

print response

In [ ]:
# Destroy the load balancer
response = elb.delete_load_balancer(
    LoadBalancerName='sample-lb'
)

print response

In [ ]:
# Delete the security group
response = ec2.delete_security_group(
    GroupId=webSGID
)

print response